# Overview

There are 4+1 steps in this model.

### Step 1: Data preparation
- Data processing: Join tables, Clean and test data, Add macro variables, Define new variables if needed.


### Step 1b: Data Visualization
- Visualize data to find insights

### Step 2: Model calibration and testing
- Run a model to get coefficient estimates and performance testing between historical and estimate CPI

### Step 3: Forecast macro variables
- Predict macro drivers which are used in step 4 to forecast CPI

### Step 4: Forecast CPI
- Forecast CPI from 4Q/2017 to 3Q/2019

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

# Load Data

In [44]:
cpi = pd.read_csv('Q2_cpi.csv', index_col='period', parse_dates = True)
frm30yr = pd.read_csv('Q2_30_Year_FRM.csv', index_col='period', parse_dates = True)
fmhpi = pd.read_csv('Q2_fmhpi.csv', index_col='period', parse_dates = True)
income = pd.read_csv('Q2_zillow_mi_market.csv', index_col='period', parse_dates = True)
hi = pd.read_csv('Q2_zillow_hi_market.csv', index_col='period', parse_dates = True)
hi_zip = pd.read_csv('Q2_zillow_hi_zip.csv', index_col='period', parse_dates = True)
ri = pd.read_csv('Q2_zillow_ri_market.csv', index_col='period', parse_dates = True)
ri_zip = pd.read_csv('Q2_zillow_ri_zip.csv', index_col='period', parse_dates = True)
mkt_pop = pd.read_csv('Q2_market_pop.csv', index_col='period', parse_dates = True)
zip_pop = pd.read_csv('Q2_zipcode_pop.csv')
zip_to_mkt = pd.read_csv('Q2_zip_to_market_corr.csv')
mkt_to_name = pd.read_csv('Q2_market_to_name.csv')

In [45]:
cpi.head()

,"Atlanta-Sandy Springs-Marietta, GA Metro Area","Boston-Cambridge-Newton, MA-NH Metro Area","Chicago-Joliet-Naperville, IL-IN-WI Metro Area","Dallas-Fort Worth-Arlington, TX Metro Area","Houston-Sugar Land-Baytown, TX Metro Area","Los Angeles-Long Beach-Anaheim, CA Metro Area","Miami-Fort Lauderdale-Miami Beach, FL Metropolitan Statistical Area","New York-Newark-Jersey City, NY-NJ-PA Metro Area","Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area","Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area"
period,,,,,,,,,,
2001-04-01,67.572311,76.393851,66.301575,70.113257,69.958842,60.952183,60.805031,63.128420,70.965044,62.413111
2001-07-01,67.292649,75.527751,66.736398,70.157962,69.702285,61.427964,60.807779,63.365518,71.602410,63.129960
2001-10-01,67.071931,76.010220,68.067210,69.743978,69.587393,61.192562,61.471180,63.758934,71.997636,63.962213
2002-01-01,67.183085,75.452801,68.453589,68.723861,69.882963,61.356233,62.187711,63.926603,71.868553,64.298790
2002-04-01,67.483460,74.662812,68.191444,67.676960,70.122951,61.038046,64.228132,64.575764,72.455305,63.792568


In [46]:
cities_cbsa = ['12060', '14460', '16980', '19100', '26420', '31080', '33100', '35620', '37980', '47900']

In [47]:
idx = pd.DatetimeIndex(start=cpi.index.min(),end=cpi.index.max(),freq='1D')
frm30yr = frm30yr.reindex(idx).interpolate(method='linear')
frm30yr = frm30yr.reindex(cpi.index)

In [48]:
mkt_nm = dict(zip(mkt_to_name.name,mkt_to_name.cbsa))

cpi = cpi.rename(columns=mkt_nm)

cpi.columns = cpi.columns.map(str)

In [49]:
# rent to home value
rv = pd.DataFrame(ri.values/hi['2010-11':].values, columns=ri.columns, index=ri.index)

## Check stationary

In [50]:
from statsmodels.tsa.stattools import adfuller

def stationary(df):
    result = adfuller(df.dropna())
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])

In [51]:
ds = [cpi['12060'], fmhpi['12060'], income['12060'], hi['12060'], ri['12060'], mkt_pop['12060'], rv['12060'], frm30yr['mortgage30us']]
for i in ds:
    print(stationary(i))

ADF Statistic: -2.768178
p-value: 0.062965
None
ADF Statistic: -1.545556
p-value: 0.510867
None
ADF Statistic: 1.224590
p-value: 0.996151
None
ADF Statistic: -3.284241
p-value: 0.015598
None
ADF Statistic: 1.045081
p-value: 0.994720
None
ADF Statistic: 0.000000
p-value: 0.958532
None
ADF Statistic: -1.530782
p-value: 0.518233
None
ADF Statistic: -1.747013
p-value: 0.407067
None


/Users/preeda/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


Essentially, most of them are non-stationary. Need to do some transformation.

## Data Transformation

In [52]:
# interpolate income and population data
income = income.resample('MS').interpolate('linear')
mkt_pop = mkt_pop.resample('MS').interpolate('linear')

In [53]:
# Make it quarterly and interpolate. Extrapolate can be improved
income = income.reindex(cpi.index)
income = income.interpolate(method='linear', fill_value='extrapolate')
mkt_pop = mkt_pop.reindex(cpi.index)
mkt_pop = mkt_pop.interpolate(method='linear', fill_value='extrapolate')

In [54]:
cpr = cpi.pct_change()
fmhpr = fmhpi.pct_change()
income_gr = income.pct_change()
hi_gr = hi.pct_change()
ri_gr = ri.pct_change()
pop_gr = mkt_pop.pct_change()
rv_chg = rv.diff()

In [55]:
frm30yr_chg = frm30yr.diff()
frm30yr_chg.columns = ['mortgage30us_chg']

Check stationary again.

In [56]:
ds = [cpr['12060'], fmhpr['12060'], income_gr['12060'], hi_gr['12060'], ri_gr['12060'], pop_gr['12060'], rv_chg['12060']]
for i in ds:
    print(stationary(i))

ADF Statistic: -2.668548
p-value: 0.079641
None
ADF Statistic: -1.988320
p-value: 0.291718
None
ADF Statistic: -2.778754
p-value: 0.061373
None
ADF Statistic: -1.723173
p-value: 0.419214
None
ADF Statistic: -4.655148
p-value: 0.000102
None
ADF Statistic: -2.217787
p-value: 0.199853
None
ADF Statistic: -2.287657
p-value: 0.175980
None


Better but there're still some issues. Due to time constraints, I'll continue to use this data to do the modeling.

### Save raw data used for step 3 Forecast macro variables

In [57]:
frm30yr.to_csv('frm30yr.csv')
mkt_pop.to_csv('mkt_pop.csv')

# Normalize data

In [58]:
def normalize(df,meannm,distnm):
    mean = df.mean().to_frame(name=meannm)
    dist = (df.max() - df.min()).to_frame(name=distnm)
    return (df.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x))), mean, dist)

In [59]:
cpr, mean_cpr, dist_cpr = normalize(cpr,'mean_cpr','dist_cpr')
fmhpr, mean_fmhpr, dist_fmhpr = normalize(fmhpr,'mean_fmhpr','dist_fmhpr')
income_gr, mean_income, dist_income = normalize(income_gr,'mean_income','dist_income')
hi_gr, mean_hi, dist_hi = normalize(hi_gr,'mean_hi','dist_hi')
ri_gr, mean_ri, dist_ri = normalize(ri_gr,'mean_ri','dist_ri')
pop_gr, mean_pop, dist_pop = normalize(pop_gr,'mean_pop','dist_pop')
rv, mean_rv, dist_rv = normalize(rv,'mean_rv','dist_rv')
frm30yr, mean_frm30yr, dist_frm30yr = normalize(frm30yr,'mean_frm30yr','dist_frm30yr')
rv_chg, mean_rv_chg, dist_rv_chg = normalize(rv_chg,'mean_rv_chg','dist_rv_chg')
frm30yr_chg, mean_frm30yr_chg, dist_frm30yr_chg = normalize(frm30yr_chg,'mean_frm30yr_chg','dist_frm30yr_chg')

In [60]:
mean_all = pd.concat([mean_cpr, mean_fmhpr, mean_income, mean_hi, mean_ri, mean_pop, mean_rv, mean_rv_chg], axis=1)
mean_all['mean_frm30yr'] = mean_frm30yr.iloc[0][0]
mean_all['mean_frm30yr_chg'] = mean_frm30yr_chg.iloc[0][0]
dist_all = pd.concat([dist_cpr, dist_fmhpr, dist_income, dist_hi, dist_ri, dist_pop, dist_rv, dist_rv_chg], axis=1)
dist_all['dist_frm30yr'] = dist_frm30yr.iloc[0][0]
dist_all['dist_frm30yr_chg'] = dist_frm30yr_chg.iloc[0][0]

In [61]:
mean_all.index.name = 'cbsa'
dist_all.index.name = 'cbsa'

In [62]:
ds = [cpr['12060'], fmhpr['12060'], income_gr['12060'], hi_gr['12060'], ri_gr['12060'], pop_gr['12060'], rv_chg['12060']]
for i in ds:
    print(stationary(i))

ADF Statistic: -2.668548
p-value: 0.079641
None
ADF Statistic: -1.988320
p-value: 0.291718
None
ADF Statistic: -2.778754
p-value: 0.061373
None
ADF Statistic: -1.723173
p-value: 0.419214
None
ADF Statistic: -4.655148
p-value: 0.000102
None
ADF Statistic: -2.217787
p-value: 0.199853
None
ADF Statistic: -2.287657
p-value: 0.175980
None


### Add lag

In [63]:
# Create lag1 and lag2 quarter
def lag_func(df):
    df_l1 = df.shift(1)
    df_l2 = df.shift(2)
    return (df_l1, df_l2)

In [64]:
cpr_l1, cpr_l2 = lag_func(cpr)
frm30yr_l1, frm30yr_l2 = lag_func(frm30yr)
fmhpr_l1, fmhpr_l2 = lag_func(fmhpr)
income_gr_l1, income_gr_l2 = lag_func(income_gr)
hi_gr_l1, hi_gr_l2 = lag_func(hi_gr)
ri_gr_l1, ri_gr_l2 = lag_func(ri_gr)
pop_gr_l1, pop_gr_l2 = lag_func(pop_gr)
rv_l1, rv_l2 = lag_func(rv)
frm30yr_chg_l1, frm30yr_chg_l2 = lag_func(frm30yr_chg)
rv_chg_l1, rv_chg_l2 = lag_func(rv_chg)

In [65]:
frm30yr_l1.columns = ['mortgage30us_l1']
frm30yr_l2.columns = ['mortgage30us_l2']
frm30yr_chg_l1.columns = ['mortgage30us_chg_l1']
frm30yr_chg_l2.columns = ['mortgage30us_chg_l2']

# Convert Data Wide to Long

In [66]:
def w_to_l(df,id1,col1,col2):
    df.reset_index(inplace=True)
    df = pd.melt(df, id_vars=[id1],value_vars=df.columns[1:])
    df.columns = [id1,col1,col2]
    df[col1] = pd.to_numeric(df[col1])
    return df

In [67]:
cpi = w_to_l(cpi,'period','cbsa','cpi')
cpr = w_to_l(cpr,'period','cbsa','ret')
cpr_l1 = w_to_l(cpr_l1,'period','cbsa','return_l1')
cpr_l2 = w_to_l(cpr_l2,'period','cbsa','return_l2')
fmhpr = w_to_l(fmhpr,'period','cbsa','fmhpr')
fmhpr_l1 = w_to_l(fmhpr_l1,'period','cbsa','fmhpr_l1')
fmhpr_l2 = w_to_l(fmhpr_l2,'period','cbsa','fmhpr_l2')
income_gr = w_to_l(income_gr,'period','cbsa','income_gr')
income_gr_l1 = w_to_l(income_gr_l1,'period','cbsa','income_gr_l1')
income_gr_l2 = w_to_l(income_gr_l2,'period','cbsa','income_gr_l2')
hi_gr = w_to_l(hi_gr,'period','cbsa','hi_gr')
hi_gr_l1 = w_to_l(hi_gr_l1,'period','cbsa','hi_gr_l1')
hi_gr_l2 = w_to_l(hi_gr_l2,'period','cbsa','hi_gr_l2')
ri_gr = w_to_l(ri_gr,'period','cbsa','ri_gr')
ri_gr_l1 = w_to_l(ri_gr_l1,'period','cbsa','ri_gr_l1')
ri_gr_l2 = w_to_l(ri_gr_l2,'period','cbsa','ri_gr_l2')
pop_gr = w_to_l(pop_gr,'period','cbsa','pop_gr')
pop_gr_l1 = w_to_l(pop_gr_l1,'period','cbsa','pop_gr_l1')
pop_gr_l2 = w_to_l(pop_gr_l2,'period','cbsa','pop_gr_l2')
rv = w_to_l(rv,'period','cbsa','rv')
rv_l1 = w_to_l(rv_l1,'period','cbsa','rv_l1')
rv_l2 = w_to_l(rv_l2,'period','cbsa','rv_l2')

In [68]:
frm30yr.reset_index(inplace=True)
frm30yr_l1.reset_index(inplace=True)
frm30yr_l2.reset_index(inplace=True)
frm30yr_chg.reset_index(inplace=True)
frm30yr_chg_l1.reset_index(inplace=True)
frm30yr_chg_l2.reset_index(inplace=True)

In [69]:
cpr.dropna(inplace=True)

In [70]:
cpr = cpr.merge(frm30yr, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(frm30yr_l1, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(frm30yr_l2, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(frm30yr_chg, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(frm30yr_chg_l1, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(frm30yr_chg_l2, how='left', left_on=['period'], right_on=['period'])
cpr = cpr.merge(cpr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(cpr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(fmhpr, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(fmhpr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(fmhpr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(income_gr, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(income_gr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(income_gr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(hi_gr, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(hi_gr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(hi_gr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(ri_gr, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(ri_gr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(ri_gr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(pop_gr, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(pop_gr_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(pop_gr_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(rv, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(rv_l1, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])
cpr = cpr.merge(rv_l2, how='left', left_on=['period','cbsa'], right_on=['period','cbsa'])

In [71]:
cpi.set_index(['period'], inplace=True)
cpr.set_index(['period'], inplace=True)

### Check Data

In [72]:
cpr.head()

,cbsa,ret,mortgage30us,mortgage30us_l1,mortgage30us_l2,mortgage30us_chg,mortgage30us_chg_l1,mortgage30us_chg_l2,return_l1,return_l2,...,hi_gr_l2,ri_gr,ri_gr_l1,ri_gr_l2,pop_gr,pop_gr_l1,pop_gr_l2,rv,rv_l1,rv_l2
period,,,,,,,,,,,,,,,,,,,,,
2001-07-01,12060,-0.048960,0.526050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.123804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-10-01,12060,-0.043616,0.408866,0.526050,NaN,-0.213258,NaN,NaN,-0.048960,NaN,...,0.074680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-01-01,12060,-0.012888,0.530169,0.408866,0.526050,0.276413,-0.213258,NaN,-0.043616,-0.048960,...,0.096688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-04-01,12060,0.004624,0.532789,0.530169,0.408866,0.032730,0.276413,-0.213258,-0.012888,-0.043616,...,0.117814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-07-01,12060,0.039185,0.375545,0.532789,0.530169,-0.295510,0.032730,0.276413,0.004624,-0.012888,...,0.160655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
cpr.shape

(650, 28)

In [74]:
cols = ['income_gr','income_gr_l1', 'income_gr_l2','ri_gr', 'ri_gr_l1', 'ri_gr_l2','rv', 'rv_l1', 'rv_l2']
cpr.drop(cols, axis=1, inplace=True)

In [75]:
cpr.isnull().sum()

cbsa                     0
ret                      0
mortgage30us             0
mortgage30us_l1         10
mortgage30us_l2         20
mortgage30us_chg        10
mortgage30us_chg_l1     20
mortgage30us_chg_l2     30
return_l1               10
return_l2               20
fmhpr                    0
fmhpr_l1                 0
fmhpr_l2                 0
hi_gr                    0
hi_gr_l1                 0
hi_gr_l2                 0
pop_gr                 150
pop_gr_l1              160
pop_gr_l2              170
dtype: int64

In [76]:
cpr.dropna(inplace=True)

In [77]:
cpr.shape

(480, 19)

In [78]:
cpr.head()

,cbsa,ret,mortgage30us,mortgage30us_l1,mortgage30us_l2,mortgage30us_chg,mortgage30us_chg_l1,mortgage30us_chg_l2,return_l1,return_l2,fmhpr,fmhpr_l1,fmhpr_l2,hi_gr,hi_gr_l1,hi_gr_l2,pop_gr,pop_gr_l1,pop_gr_l2
period,,,,,,,,,,,,,,,,,,,
2005-10-01,12060,0.137064,0.210812,0.109352,0.235522,0.235671,-0.231707,0.150728,0.029383,0.051247,0.040290,0.023973,0.024726,0.118606,0.098996,0.099571,0.485892,0.495601,0.505618
2006-01-01,12060,0.179827,0.285691,0.210812,0.109352,0.181092,0.235671,-0.231707,0.137064,0.029383,-0.074971,-0.048408,0.047618,0.096583,0.097139,0.117867,0.476477,0.485892,0.495601
2006-04-01,12060,0.160063,0.326874,0.285691,0.210812,0.111908,0.181092,0.235671,0.179827,0.137064,0.130991,0.102689,0.033318,0.075392,0.075791,0.076194,0.133119,0.476477,0.485892
2006-07-01,12060,0.294379,0.434324,0.326874,0.285691,0.247970,0.111908,0.181092,0.160063,0.179827,-0.036117,0.070970,0.134314,0.093641,0.074536,0.094633,0.131336,0.133119,0.476477
2006-10-01,12060,0.178872,0.309277,0.434324,0.326874,-0.229401,0.247970,0.111908,0.294379,0.160063,-0.038141,-0.025233,-0.073418,0.034253,0.092589,0.093113,0.129577,0.131336,0.133119


In [79]:
cpr.corr()

,cbsa,ret,mortgage30us,mortgage30us_l1,mortgage30us_l2,mortgage30us_chg,mortgage30us_chg_l1,mortgage30us_chg_l2,return_l1,return_l2,fmhpr,fmhpr_l1,fmhpr_l2,hi_gr,hi_gr_l1,hi_gr_l2,pop_gr,pop_gr_l1,pop_gr_l2
cbsa,1.000000e+00,-0.040757,-3.104879e-17,-3.525410e-17,5.034293e-18,2.624937e-18,1.037545e-17,0.000000,-0.027319,-0.013477,-0.111540,-0.104456,-0.111582,-0.120019,-0.112296,-0.106991,-0.002172,0.003576,0.008609
ret,-4.075655e-02,1.000000,-1.202622e-01,-1.956894e-01,-2.771698e-01,2.124903e-01,2.350100e-01,0.190960,0.827045,0.691240,0.320601,0.334657,0.379748,0.540109,0.565761,0.572889,0.110954,0.106885,0.146577
mortgage30us,-3.104879e-17,-0.120262,1.000000e+00,9.376492e-01,8.784959e-01,1.833078e-01,1.422138e-01,0.049472,-0.044324,0.015334,-0.358215,-0.311517,-0.286601,-0.425100,-0.403690,-0.364537,0.195034,0.228436,0.302894
mortgage30us_l1,-3.525410e-17,-0.195689,9.376492e-01,1.000000e+00,9.366267e-01,-1.698151e-01,1.524791e-01,0.147723,-0.095671,-0.025485,-0.448960,-0.418239,-0.390821,-0.496522,-0.472377,-0.435346,0.177129,0.207280,0.237797
mortgage30us_l2,5.034293e-18,-0.277170,8.784959e-01,9.366267e-01,1.000000e+00,-1.582915e-01,-2.034166e-01,0.163726,-0.160606,-0.071316,-0.497005,-0.474366,-0.451637,-0.565385,-0.540085,-0.500228,0.171130,0.215037,0.240460
mortgage30us_chg,2.624937e-18,0.212490,1.833078e-01,-1.698151e-01,-1.582915e-01,1.000000e+00,-2.804247e-02,-0.277533,0.144914,0.115556,0.254157,0.299667,0.292766,0.199038,0.191455,0.197727,0.051999,0.061424,0.186221
mortgage30us_chg_l1,1.037545e-17,0.235010,1.422138e-01,1.524791e-01,-2.034166e-01,-2.804247e-02,1.000000e+00,-0.049031,0.185704,0.129961,0.147348,0.169338,0.181846,0.207330,0.203432,0.194488,0.012262,-0.027335,-0.013767
mortgage30us_chg_l2,0.000000e+00,0.190960,4.947219e-02,1.477226e-01,1.637256e-01,-2.775333e-01,-4.903122e-02,1.000000,0.249987,0.197797,-0.164011,-0.127291,0.018175,0.131199,0.180251,0.209579,-0.060759,0.045310,0.005376
return_l1,-2.731944e-02,0.827045,-4.432381e-02,-9.567066e-02,-1.606060e-01,1.449136e-01,1.857041e-01,0.249987,1.000000,0.828103,0.277960,0.274657,0.306687,0.474337,0.507030,0.523838,0.122562,0.163060,0.156806
return_l2,-1.347720e-02,0.691240,1.533448e-02,-2.548544e-02,-7.131559e-02,1.155561e-01,1.299609e-01,0.197797,0.828103,1.000000,0.222322,0.243635,0.276161,0.406804,0.435975,0.452572,0.098438,0.158645,0.195765


# Final dataset

In [80]:
cpr.to_csv('cpr_final.csv')
cpi.to_csv('cpi_final.csv')
mean_all.to_csv('mean_all.csv')
dist_all.to_csv('dist_all.csv')

In [81]:
cpr.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cbsa,480.0,27470.000000,11088.217298,12060.000000,16980.000000,28750.000000,35620.000000,47900.000000
ret,480.0,-0.018687,0.202388,-0.703909,-0.101883,0.004099,0.111199,0.475699
mortgage30us,480.0,-0.093392,0.270243,-0.467211,-0.302198,-0.173126,0.199300,0.434324
mortgage30us_l1,480.0,-0.084313,0.269579,-0.467211,-0.297892,-0.157963,0.199300,0.434324
mortgage30us_l2,480.0,-0.073963,0.272116,-0.467211,-0.297892,-0.150475,0.216990,0.434324
mortgage30us_chg,480.0,0.008708,0.195707,-0.516035,-0.123511,-0.030176,0.133432,0.483965
mortgage30us_chg_l1,480.0,0.006098,0.198051,-0.516035,-0.125625,-0.030176,0.133432,0.483965
mortgage30us_chg_l2,480.0,0.010238,0.198954,-0.516035,-0.125625,-0.014417,0.142945,0.483965
return_l1,480.0,-0.015818,0.204392,-0.703909,-0.101883,0.008345,0.113015,0.475699
return_l2,480.0,-0.014573,0.204748,-0.703909,-0.101883,0.011754,0.116365,0.475699
